In [50]:
import sys
import re
import random
from operator import itemgetter
import urlparse
from posixpath import split, splitext, normpath
from collections import defaultdict, Counter
from random import sample
from itertools import chain


ImportError: No module named urlib

In [115]:
def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [59]:
def read_lines(file_name):
    with open(file_name) as f:
        lines = f.readlines()
        lines = [x.strip() for x in lines] 
    return lines


In [3]:
general_urls = read_lines('data/urls.lenta.general')
examined_urls = read_lines('data/urls.lenta.examined')

In [4]:
general_urls

['http://lenta.ru/news/2010/07/07/widow/',
 'http://lenta.ru/news/2010/03/22/trans/',
 'http://lenta.ru/news/2013/08/18/brotherhood/',
 'http://lenta.ru/world/1999/09/10/mtv/',
 'http://lenta.ru/news/2009/09/24/hilton/',
 'http://lenta.ru/news/2010/06/27/stalin/picture.jpg',
 'http://lenta.ru/vybory/2004/08/22/',
 'http://lenta.ru/news/2014/02/03/',
 'http://lenta.ru/russia/2004/10/10/enemy/',
 'http://lenta.ru/news/2010/06/08/more/',
 'http://lenta.ru/sport/2001/07/15/formulaone/',
 'http://lenta.ru/news/2010/10/07/imya/',
 'http://lenta.ru/news2/2009/01/11/mmk/',
 'http://lenta.ru/world/2002/05/30/cheney/',
 'http://lenta.ru/news/2009/09/12/mine/picture.jpg',
 'http://lenta.ru/news/2010/07/05/cuts/_Printed.htm',
 'http://lenta.ru/articles/2010/07/14/museum/',
 'http://lenta.ru/news/2009/01/12/molotov/%0D',
 'http://lenta.ru/world/2003/03/13/lab/',
 'http://lenta.ru/russia/2004/06/18/fire1/_Printed.htm',
 'http://lenta.ru/news/2008/10/13/pizza',
 'http://lenta.ru/www.youtube.com/embed

In [5]:
url = 'http://lenta.ru/news/2008/10/01/pseudoscience/_Printed.htm'

In [6]:
def path_to_segments(path):
#path = '/news/2010/07/07/widow/'
    tmp = path
    tmp = normpath(tmp)
    segments = []
    while tmp != "/":
        (tmp, last) = split(tmp)
        #print (tmp, last), '\n'
        segments.insert(0, last)
    return segments

In [7]:
path = urlparse.urlparse(url).path
segments = path_to_segments(path)
segments
#segments.append('a102b.htm')

['news', '2008', '10', '01', 'pseudoscience', '_Printed.htm']

In [116]:
numeric_substr = re.compile('[^\d]+\d+[^\d]+$')

In [117]:
def segments_to_features(segments):
    features = []
    
    # 1
    features.append('segments:' + str(len(segments)))
                    
    for index, string in enumerate(segments):
        # 4a
        features.append('segment_name_' + str(index) + ':' + str(string))
        # 4b
        if is_number(string):
            features.append('segment_[0-9]_' + str(index) + ':' + '1')
        # 4d
        extension = splitext(string)[1][1:]
        if extension:
            features.append('segment_ext_' + str(index) + ':' + extension)
        # 4c    
        string = urlparse.unquote(string)
        #numeric_substr = re.compile('[^\d]+\d+[^\d]+$')
        if numeric_substr.match(string):
            features.append('segment_substr[0-9]_' + str(index) + ':' + '1')
            # 4e
            if extension:
                features.append('segment_ext_substr[0-9]_' + str(index) + ':' + extension)
        # 4f
        features.append('segment_len_' + str(index) + ':' + str(len(string)))      
        
    return features

In [118]:
def all_segments_to_features(segments):
    features = []
    
    # 1
    #features.append('segments:' + str(len(segments)))
                    
    for index, string in enumerate(segments):
        # 4a
        features.append('segment_name_' + str(index) + ':' + str(string))
        # 4b
        if is_number(string):
            features.append('segment_[0-9]_' + str(index) + ':' + '1')
        # 4d
        extension = splitext(string)[1][1:]
        if extension:
            features.append('segment_ext_' + str(index) + ':' + extension)
        # 4c    
        string = urlparse.unquote(string)
        if numeric_substr.match(string):
            features.append('segment_substr[0-9]_' + str(index) + ':' + '1')
            # 4e
            if extension:
                features.append('segment_ext_substr[0-9]_' + str(index) + ':' + extension)
        # 4f
        features.append('segment_len_' + str(index) + ':' + str(len(string)))      
        
    return features

In [69]:
segments_to_features(segments)

['segments:6',
 'segment_name_0:news',
 'segment_len_0:4',
 'segment_name_1:2008',
 'segment_[0-9]_1:1',
 'segment_len_1:4',
 'segment_name_2:10',
 'segment_[0-9]_2:1',
 'segment_len_2:2',
 'segment_name_3:01',
 'segment_[0-9]_3:1',
 'segment_len_3:2',
 'segment_name_4:pseudoscience',
 'segment_len_4:13',
 'segment_name_5:_Printed.htm',
 'segment_ext_5:htm',
 'segment_len_5:12']

In [10]:
url = urlparse.urlparse('http://lenta.ru/PKNnZ/news/2012/06/10/krasn2/?TB_iframe=true&height=650&width=850')

In [11]:
def query_to_params_features(query):
    features = []
    if query:
        parameters = dict(urlparse.parse_qsl(query))
        # 2
        features.append('param_name:' + str(parameters.keys()))
        for parameter, value in parameters.iteritems():
            # 3
            features.append('param:' + str(parameter) + '=' + str(value))   
    return features

In [12]:
query_to_params_features(url.query)

["param_name:['TB_iframe', 'width', 'height']",
 'param:TB_iframe=true',
 'param:width=850',
 'param:height=650']

In [72]:
def url_list_to_features(url_list):
    features = defaultdict(list)
    for url in url_list:
        parsed_url = urlparse.urlparse(url)
        segments = path_to_segments(parsed_url.path)
        features[url] += segments_to_features(segments)
        features[url] += query_to_params_features(parsed_url.query)
    return features
    

In [97]:
def url_list_to_all_features(url_list):
    features = []
    for url in url_list:
        parsed_url = urlparse.urlparse(url)
        segments = path_to_segments(parsed_url.path)
        features += segments_to_features(segments)
        features += query_to_params_features(parsed_url.query)
    return features
    

In [14]:
def frequency(dictionary_of_lists, threshold):
    frequencies = Counter(chain.from_iterable(dictionary_of_lists.values()))
    frequencies = {feature:count for feature, count in frequencies.iteritems() if count > threshold}
    return sorted(frequencies.items(), key=itemgetter(1), reverse=True)

In [25]:
def write_features(file_name, features):
    with open(file_name, 'w') as f:
        f.write('\n'.join('%s\t%s' % x for x in features))

In [124]:
def extract_features(INPUT_FILE_1, INPUT_FILE_2, OUTPUT_FILE):
    
    examined_urls = read_lines(INPUT_FILE_1)
    general_urls = read_lines(INPUT_FILE_2)
    
    size = len(examined_urls) / 2
    examined_urls = sample(examined_urls, size)
    general_urls = sample(general_urls, size)
    
    #features = url_list_to_features(examined_urls + general_urls)    
    features = url_list_to_all_features(examined_urls + general_urls) 
    
    #frequency
    alpha = 0.1
    threshold = size * alpha
    #write_features(OUTPUT_FILE, frequency(features, threshold))
    frequencies = Counter(features)
    frequencies = {feature:count for feature, count in frequencies.iteritems() if count > threshold}
    frequencies = sorted(frequencies.items(), key=itemgetter(1), reverse=True)
    write_features(OUTPUT_FILE, frequencies)
    #return features
    #print >> sys.stderr, "Not implemented"


In [125]:
wiki_features = extract_features('data/urls.wikipedia.examined', 'data/urls.wikipedia.general', 'check.fea')

In [123]:
wiki_features

In [122]:
Counter(chain.from_iterable(wiki_features.values())).most_common()

AttributeError: 'NoneType' object has no attribute 'values'

In [64]:
examined_urls = read_lines('data/urls.wikipedia.examined')

In [65]:
numeric_substr = re.compile('[^\d]+\d+[^\d]+$')
if numeric_substr.match(urlparse.unquote(string)):
    features.append('segment_substr[0-9]_' + str(index) + ':' + '1')

NameError: name 'string' is not defined

In [ ]:
filter(r.match, list)

In [ ]:
if numeric_substr.match(string):
    features.append('segment_substr[0-9]_' + str(index) + ':' + '1')